In [ ]:
#Importing the Libraries
import timeit
import os
import psutil
import math
from heapq import *

#Constant
GOAL_STATE = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "0"]

# Node class representing the state of the puzzle, its parent, action, and cost
class Node:
    def __init__(self, state, parent, action):
        self.state = state
        self.parent = parent
        self.action = action
        self.cost = 0 if self.parent is None else parent.cost + 1

    def __lt__(self, alternate):
        return (self.state.board < alternate.state.board) and (self.cost < alternate.cost)

    def __eq__(self, alternate):
        return self.state.board == alternate.state.board

    def __repr__(self):
        return str(self.state.board)

# Environment class for managing the board and performing actions
class Environment:
    def __init__(self, board):
        self.side = 4
        self.board = board

    # Perform the specified action on the current board, and return the resulting board
    def perform_action(self, action):
        new_board = self.board[:]
        zero_index = new_board.index('0')

        if action == "L" and (zero_index % 4) > 0:
            new_board[zero_index - 1], new_board[zero_index] = new_board[zero_index], new_board[zero_index - 1]
        if action == "R" and (zero_index % 4) < 3:
            new_board[zero_index + 1], new_board[zero_index] = new_board[zero_index], new_board[zero_index + 1]
        if action == "U" and (zero_index // 4) >= 1:
            new_board[zero_index - 4], new_board[zero_index] = new_board[zero_index], new_board[zero_index - 4]
        if action == "D" and (zero_index // 4) < 3:
            new_board[zero_index + 4], new_board[zero_index] = new_board[zero_index], new_board[zero_index + 4]
        return Environment(new_board)

#Calculate the Manhattan distance for the given node
def manhattan_distance(node):
    board = node.state.board
    side = node.state.side
    manhattan_distance = 0

    for i in range(len(board)):
        value = int(board[i])
        if value == 0:
            continue
        current_x, current_y = i // 4, i % 4
        goal_x, goal_y = (value - 1) // 4, (value - 1) % 4\
        manhattan_distance += abs(goal_x - current_x) + abs(goal_y - current_y)\
    return manhattan_distance

# Calculate the number of misplaced tiles for the given node
def misplaced_tiles(node):
def misplaced_tiles(node):
    board = node.state.board
    misplaced_tiles = sum(1 for i in range(1, len(board)) if i != int(board[i - 1]))
    return misplaced_tiles

# A* search function using the specified heuristic function
def astar_search(initial_node, heuristic_function):
    heuristic_name = heuristic_function.__name__.replace('_', ' ').title()
    print(f"--------------- {heuristic_name} ---------------")

    start_time = timeit.default_timer()
    initial_memory_state = psutil.Process(os.getpid()).memory_info().rss / 1024.0

    queue_nodes = []
    visited_nodes = []
    # Push the initial node with its heuristic value onto the queue
    heappush(queue_nodes, (initial_node.cost + heuristic_function(initial_node), initial_node))
    move_counter = 0
    
    # Loop until the queue is empty
    while queue_nodes:
        current_node = heappop(queue_nodes)[1]
        move_counter += 1
        visited_nodes.append(current_node)

        # Check if the current node is the goal state
        if current_node.state.board == GOAL_STATE:
            path = []
            while current_node.parent is not None:
                path.append(current_node.action)
                current_node = current_node.parent
            path.reverse()

            print("Moves:", str(path))
            print("Number of Nodes expanded:", move_counter)

            final_memory_state = psutil.Process(os.getpid()).memory_info().rss / 1024.0
            memory_used = final_memory_state - initial_memory_state
            print(f"Memory Used: {memory_used} Kilobytes")

            end_time = timeit.default_timer()
            time_elapsed = round((end_time - start_time) * 1000, 3)
            print(f"Time Taken: {time_elapsed} ms")
            return path, move_counter
        else:
            # Generate child nodes
            children = []
            actions = ["U", "D", "L", "R"]

            for action in actions:
                child_state = current_node.state.perform_action(action)
                child_node = Node(child_state, current_node, action)
                children.append(child_node)

            for child in children:
                if child in visited_nodes:
                    continue
                else:
                    heappush(queue_nodes, (child.cost + heuristic_function(child), child))
    print("Solution Could Not be Found")
    return False

def main():
    initial_state = input("Enter the Initial State of the Puzzle: ")
    if len(initial_state) < 37:
        print("Insufficient Data. Please Try Again")
        time.sleep(5)
        exit()

    if initial_state == "1 2 3 4 5 6 7 8 9 10 11 12 13 15 14 0":
        print("Impossible to Compute")
        time.sleep(5)
        exit()

    initial_state = initial_state.split(" ")
    initial_node = Node(Environment(initial_state), None, None)

    manhattan_output = astar_search(initial_node, manhattan_distance)
    misplaced_tiles_output = astar_search(initial_node, misplaced_tiles)

if __name__=="__main__":
    main()